In [4]:
import json
from bs4 import BeautifulSoup
import requests

In [29]:
# các url cần thiết
PTIT_PORTAL_URL = "https://portal.ptit.edu.vn"
THONG_BAO_URL = 'https://ptit.edu.vn/tin-tuc-su-kien/thong-bao'
MAX_PAGE = 4

In [37]:
def get_content_from_a_notice(url):
    '''
    Hàm này trả về nội dung của một thông báo bao gồm cả văn bản và hình ảnh
    tham số:
        url: url của thông báo
    return:
        dict: 
            {
                "text": str,
                "image_links": list[str]
            }
    '''
    response = requests.get(url)
    soup = BeautifulSoup(response.text, features="html.parser")
    content = soup.find("div", class_="post-content")
    # nếu không tìm thấy content thì trả về None
    if not content:
        return None
    text = '' # văn bản
    image_links = [] # các link ảnh
    for p in content.find_all("p"):
        # nếu có text thì thêm vào text
        if p.text.strip() :
            text += p.text.strip() + '\n'
    # nếu có ảnh thì thêm vào image_links
    for img in content.find_all("img", src=True):
        image_links.append(img['src'])
    return dict(text=text, image_links=image_links)

def Craw_notices(url):
    '''
    Hàm này trả về thông báo từ trang thông báo của trường PTIT
    return:
        list[dict]: 
            [
                {
                    "title": str,
                    "date": str,
                    "url": str,
                    "content": dict
                }
            ]
    '''
    data = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, features="html.parser")
    ul = soup.find("ul", class_="ova-blog column_4 version_1 default-post")
    items = ul.find_all("li", class_="item")
    # lấy thông tin từ từng item
    for item in items:
        content = item.find("div", class_="content")
        # lấy title, url, date từ item
        title = content.find("h2", class_= 'post-title').text.strip()
        url = content.find("a" )['href'].strip()
        date = content.find("span", class_="right date").text.strip()
        data.append(dict(title=title, date=date, url=url, content=get_content_from_a_notice(url)))
    return data

In [38]:
notices = []
for i in range(1, MAX_PAGE + 1):
    URL_BASE = f"{THONG_BAO_URL}/page/{i}"
    notices.extend(Craw_notices(URL_BASE))

In [45]:
with open(r'.\Results\notices.json', 'w', encoding='utf-8') as f:
    s = json.dumps(notices, ensure_ascii=False, indent=4)
    f.write(s)